**Install Required Libraries**

In [3]:
!pip install transformers datasets


**upload dataset file**

In [5]:
from google.colab import files
uploaded = files.upload()


Saving exercise_dataset.csv to exercise_dataset (1).csv


In [6]:
import pandas as pd

**csv file convert into  txt**

In [7]:
df = pd.read_csv("exercise_dataset.csv")

with open("fitness_chat_data.txt", "w", encoding="utf-8") as f:
    for i, row in df.iterrows():
        question = (f"My age is {row['Age']}, I do Exercise {row['Exercise']}. "
                    f"My heart rate is {row['Heart Rate']} and my BMI is {row['BMI']:.1f}. "
                    f"How many calories will I burn?")
        answer = f"You will burn approximately {row['Calories Burn']:.0f} calories."

        f.write(f"You: {question}\n")
        f.write(f"Bot: {answer}\n\n")


**read txt file**

In [8]:

with open("fitness_chat_data.txt", "r", encoding="utf-8") as f:
    for _ in range(5):
        print(f.readline())

You: My age is 45, I do Exercise Exercise 2. My heart rate is 170 and my BMI is 29.4. How many calories will I burn?

Bot: You will burn approximately 287 calories.



You: My age is 25, I do Exercise Exercise 7. My heart rate is 142 and my BMI is 21.3. How many calories will I burn?

Bot: You will burn approximately 343 calories.



**install torch**


In [9]:
!pip install torch

**import libraries**

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

**Load Pretrained GPT-2**

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Tokenize Dataset**

In [12]:
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

dataset = load_dataset("fitness_chat_data.txt", tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


**Training Arguments Define**

In [22]:
training_args = TrainingArguments(
    output_dir="./gpt2-fitness-model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    report_to="none"             #wandh handle
)


In [20]:
!pip install wandb

**fine tuning**

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,0.545400
1000,0.501600
1500,0.495900
2000,0.490300


TrainOutput(global_step=2043, training_loss=0.5080150934743111, metrics={'train_runtime': 236.2324, 'train_samples_per_second': 17.297, 'train_steps_per_second': 8.648, 'total_flos': 266909810688000.0, 'train_loss': 0.5080150934743111, 'epoch': 3.0})

if user  give double word like Exercise Exercise change one word  Exercise

In [29]:
import re

def clean_input(text):

    return re.sub(r'\b(\w+)\s+\1\b', r'\1', text, flags=re.IGNORECASE)

In [32]:
def chat_with_bot():
    print("Fitness Chatbot (type 'exit' to end):\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Bot: Goodbye! Stay healthy!")
            break
        cleaned_input = clean_input(user_input)

        prompt = f"User: {cleaned_input}\nBot:"

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
        )

        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        bot_reply = decoded_output[len(prompt):].strip()
        bot_reply = bot_reply.split('\n')[0]

        print("Bot:", bot_reply)




**call function**

In [33]:
chat_with_bot()

Fitness Chatbot (type 'exit' to end):

You: My age is 43, I do Exercise 6. My heart rate is 102 and my BMI is 23.5.
Bot: You will burn approximately 498 calories.
You: My age is 25, I do Exercise Exercise 7. My heart rate is 142 and my BMI is 21.3. How many calories will I burn?
Bot: You will burn approximately 200 calories.
You: My age is 20, I do  Exercise 7. My heart rate is 142 and my BMI is 21.3. How many calories will I burn?
Bot: You will burn approximately 494 calories.
You: exit
Bot: Goodbye! Stay healthy!
